# Smoothness as a function of # training samples

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
ms = pd.read_feather("./0406_mnist_binary_increasing/measures.feather")
smooth.analysis.remove_constant_columns(ms, verbose=True)

ms.head()

In [ ]:
def plot_measure(ms, measure_name):
    grid = sns.relplot(
        data=ms,
        x="dataset.samples_train",
        y=measure_name,
        hue="model.hidden_size",
        palette=smooth.analysis.make_palette(ms["model.hidden_size"]),
        kind="line",
    )

    ax = grid.axes[0][0]
    if "loss_train" in measure_name:
        ax.set_yscale("log")
    ax.set_xscale("log")
#     ax.set_xlim((3, 1200))
    plt.show()

for measure in [
    "loss_train", "loss_test",
#     "gradient_norm_train",
    "gradient_norm_test",
    "weights_product",
    "path_length_f_test",
    "path_length_d_test",
]:
    plot_measure(ms, measure)